# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [558]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.pipeline import make_pipeline

### 데이터 셋 읽어오기

In [559]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

train_test_data = [df_train, df_test]

In [560]:
df_train.head(10) # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
5,1.00,/Abuja/Nigeria,AS,0.040816,16328,End-Customer,SMB,NaN,NaN,NaN,...,LGEAF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,5,True
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,...,LGESJ,NaN,1,0,0.003079,0.026846,corporate / office,Engineering,6,True
7,1.00,/Guwahati/India,AS,0.088889,17983,End-Customer,SMB,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,7,True
8,0.75,/Cebu city/Philippines,AS,0.066667,30867,Specifier/ Influencer,Enterprise,3.0,NaN,NaN,...,LGEPH,less than 3 months,0,0,0.003079,0.026846,corporate / office,Construction,8,True
9,0.75,"/hauz khas,delhi/India",AS,0.088889,6084,End-Customer,SMB,NaN,NaN,NaN,...,LGEIL,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,NaN,9,True


In [561]:
df_test.head() # 테스트 데이터 살펴보기

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False


## 2. 데이터 전처리

### 레이블 인코딩

In [562]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [563]:
# columns = list(df_train.columns)
# data = pd.concat([df_train[columns], df_test[columns]])
# data.tail(50)

In [564]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

# print(df_train.isna().sum()/len(df_train))

columns = list(df_train.columns)

for dataset in train_test_data: 
    mode_fill = dataset.groupby('lead_owner')['customer_country'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    dataset['customer_country'] = dataset['customer_country'].fillna(mode_fill)

    mode_fill = dataset.groupby('lead_owner')['customer_country.1'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    dataset['customer_country.1'] = dataset['customer_country.1'].fillna(mode_fill)

    dataset.dropna(subset=['customer_country', 'customer_country.1'], inplace=True)

    mode_fill = dataset.groupby('customer_position')['inquiry_type'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    dataset['inquiry_type'] = dataset['inquiry_type'].fillna(mode_fill)

    mode_fill = dataset.groupby('customer_job')['customer_type'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    dataset['customer_type'] = dataset['customer_type'].fillna(mode_fill)

    mode_fill_remaining = dataset['customer_type'].mode().iloc[0]
    dataset['customer_type'] = dataset['customer_type'].fillna(mode_fill_remaining)

    mode_fill = dataset.groupby('customer_type')['inquiry_type'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    dataset['inquiry_type'] = dataset['inquiry_type'].fillna(mode_fill)

    dataset['com_reg_ver_win_rate'] = pd.to_numeric(dataset['com_reg_ver_win_rate'], errors='coerce')
    average_win_rate = dataset['com_reg_ver_win_rate'].mean(skipna=True)
    dataset['com_reg_ver_win_rate'].fillna(average_win_rate, inplace=True)

    dataset['historical_existing_cnt'] = pd.to_numeric(dataset['historical_existing_cnt'], errors='coerce')
    average_win_rate = dataset['historical_existing_cnt'].mean(skipna=True)
    dataset['historical_existing_cnt'].fillna(average_win_rate, inplace=True)

    dataset['ver_win_rate_x'] = pd.to_numeric(dataset['ver_win_rate_x'], errors='coerce')
    average_win_rate = dataset['ver_win_rate_x'].mean(skipna=True)
    dataset['ver_win_rate_x'].fillna(average_win_rate, inplace=True)

    dataset['ver_win_ratio_per_bu'] = pd.to_numeric(dataset['ver_win_ratio_per_bu'], errors='coerce')
    average_win_rate = dataset['ver_win_ratio_per_bu'].mean(skipna=True)
    dataset['ver_win_ratio_per_bu'].fillna(average_win_rate, inplace=True)

    # Count the number of missing values
    missing_values_count = dataset['customer_country'].isnull().sum()
    print("Number of missing values in customer_country:", missing_values_count)

    missing_values_count = dataset['customer_country.1'].isnull().sum()
    print("Number of missing values in customer_country.1:", missing_values_count)

    missing_values_count = dataset['inquiry_type'].isnull().sum()
    print("Number of missing values in inquiry_type: ", missing_values_count)

    missing_values_count = dataset['com_reg_ver_win_rate'].isnull().sum()
    print("Number of missing values in com_reg_ver_win_rate: ", missing_values_count)

    missing_values_count = dataset['historical_existing_cnt'].isnull().sum()
    print("Number of missing values in historical_existing_cnt: ", missing_values_count)

    missing_values_count = dataset['customer_type'].isnull().sum()
    print("Number of missing values in customer_type: ", missing_values_count)

    missing_values_count = dataset['ver_win_rate_x'].isnull().sum()
    print("Number of missing values in ver_win_rate_x: ", missing_values_count)

    missing_values_count = dataset['ver_win_ratio_per_bu'].isnull().sum()
    print("Number of missing values in ver_win_ratio_per_bu: ", missing_values_count)


df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

Number of missing values in customer_country: 0
Number of missing values in customer_country.1: 0
Number of missing values in inquiry_type:  0
Number of missing values in com_reg_ver_win_rate:  0
Number of missing values in historical_existing_cnt:  0
Number of missing values in customer_type:  0
Number of missing values in ver_win_rate_x:  0
Number of missing values in ver_win_ratio_per_bu:  0
Number of missing values in customer_country: 0
Number of missing values in customer_country.1: 0
Number of missing values in inquiry_type:  0
Number of missing values in com_reg_ver_win_rate:  0
Number of missing values in historical_existing_cnt:  0
Number of missing values in customer_type:  0
Number of missing values in ver_win_rate_x:  0
Number of missing values in ver_win_ratio_per_bu:  0


In [565]:
data.tail(50)

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner
5221,0.25,/ League City / United States,AS,0.092855,47466,End Customer,Enterprise,11.000000,NaN,NaN,...,none,LGEUS,NaN,0,0,0.001114,0.050295,NaN,NaN,730
5222,0.25,ss129jf / CA / United States,AS,0.092855,41098,End-Customer,SMB,20.028876,NaN,NaN,...,none,LGEUS,NaN,0,0,0.001114,0.050295,NaN,NaN,731
5223,0.25,/Noida/India,AS,0.092855,13414,End-Customer,SMB,20.028876,NaN,NaN,...,none,LGEIL,NaN,0,0,0.001114,0.050295,NaN,NaN,9
5224,1.00,/ OAXACA / Mexico,ID,0.124122,21715,End Customer,SMB,20.028876,1.0,NaN,...,manager,LGEMS,less than 3 months,1,0,0.000717,0.071345,hotel & accommodation,Hotel,263
5225,0.50,/ / India,ID,0.092855,36591,End Customer,Enterprise,20.028876,NaN,NaN,...,none,LGEIL,less than 3 months,0,0,0.001114,0.050295,NaN,NaN,174
5226,1.00,/ GUAYAS/MILAGRO / Panama,ID,0.074949,23254,End Customer,Enterprise,20.028876,1.0,NaN,...,ceo/founder,LGEPS,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,192
5227,0.25,10990 ROE AVE / KS / United States,AS,0.092855,46559,End-Customer,SMB,20.028876,NaN,NaN,...,none,LGEUS,NaN,0,0,0.001114,0.050295,NaN,NaN,731
5228,0.75,/ JOÃO PESSOA / Brazil,AS,0.092855,19357,Specifier/ Influencer,Enterprise,20.028876,NaN,NaN,...,director,LGESP,less than 3 months,0,0,0.001114,0.050295,NaN,NaN,97
5229,0.00,/ / Brazil,ID,0.092855,9935,End Customer,Enterprise,20.028876,NaN,NaN,...,none,LGESP,NaN,0,0,0.001114,0.050295,NaN,NaN,278
5230,1.00,/ São José do Rio Preto - SP / Brazil,ID,0.124122,36363,End Customer,SMB,20.028876,1.0,NaN,...,associate/analyst,LGESP,3 months ~ 6 months,1,1,0.000717,0.071345,hotel & accommodation,Dormitory,572


In [566]:
df_test.head(50)

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.000000,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,0.102336,5405,End Customer,SMB,21.272941,NaN,...,LGEUS,NaN,0,0,0.000013,0.058388,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,0.102336,13597,Specifier/ Influencer,SMB,21.272941,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,End Customer,Enterprise,21.272941,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.000000,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False
5,17528,0.00,/ / Germany,IT,0.102336,22534,End Customer,SMB,21.272941,NaN,...,LGEDG,NaN,0,0,0.001093,0.058388,NaN,NaN,429,False
6,2853,1.00,/ / Egypt,ID,0.102336,32761,End Customer,Enterprise,21.272941,NaN,...,LGEEG,6 months ~ 9 months,0,0,0.001093,0.058388,NaN,NaN,234,False
7,3061,1.00,/ / France,ID,0.102336,46010,Service Partner,Enterprise,21.272941,NaN,...,LGEFS,3 months ~ 6 months,0,0,0.001093,0.058388,NaN,NaN,577,False
8,2909,0.75,/ /,ID,0.102336,2452,End Customer,Enterprise,0.000000,NaN,...,LGEUS,3 months ~ 6 months,0,0,0.001093,0.058388,NaN,NaN,310,False
9,9577,0.25,Education / / UNITED STATES,ID,0.833333,26303,End Customer,Enterprise,32.000000,NaN,...,LGEUS,NaN,0,0,0.000013,0.053571,transportation,Others,353,False


다시 학습 데이터와 제출 데이터를 분리합니다.

In [567]:

for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]



### 피쳐의 중요도 확인하기

### 상관계수 : 필요없는 특성 제거

In [568]:
# drop_column = list(df_train.columns)
# print(drop_column)

# # 상관계수 계산
# corr_matrix = df_train.corr()
# corr_matrix["is_converted"].sort_values(ascending=False)
    

### is_converted 와 가장 관련 없는 칼럼을 지운다.


In [569]:
df_train = df_train.drop(columns=['expected_timeline', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver'])
df_test = df_test.drop(columns=['expected_timeline', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver'])
df_train.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,customer_position,response_corporate,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,9070,0,0.066667,32160,10,0,19.913559,420,62,...,47,33,1,0,0.003079,0.026846,0,28,0,True
1,1.0,8406,0,0.066667,23122,10,0,12.000000,303,96,...,21,33,1,0,0.003079,0.026846,0,0,1,True
2,1.0,6535,0,0.088889,1755,10,0,144.000000,160,56,...,80,21,1,0,0.003079,0.026846,0,17,2,True
3,1.0,3388,0,0.088889,4919,10,0,19.913559,166,44,...,21,21,1,0,0.003079,0.026846,0,44,3,True
4,1.0,5799,0,0.088889,17126,29,0,19.913559,84,97,...,80,21,0,0,0.003079,0.026846,0,86,4,True


### 2-2. 학습, 검증 데이터 분리

In [570]:
df_train.fillna(df_train.mean())

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,customer_position,response_corporate,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,9070,0,0.066667,32160,10,0,19.913559,420,62,...,47,33,1,0,0.003079,0.026846,0,28,0,True
1,1.00,8406,0,0.066667,23122,10,0,12.000000,303,96,...,21,33,1,0,0.003079,0.026846,0,0,1,True
2,1.00,6535,0,0.088889,1755,10,0,144.000000,160,56,...,80,21,1,0,0.003079,0.026846,0,17,2,True
3,1.00,3388,0,0.088889,4919,10,0,19.913559,166,44,...,21,21,1,0,0.003079,0.026846,0,44,3,True
4,1.00,5799,0,0.088889,17126,29,0,19.913559,84,97,...,80,21,0,0,0.003079,0.026846,0,86,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,10287,0,0.091691,33747,9,1,19.913559,160,200,...,47,34,0,0,0.000026,0.028777,8,62,694,False
59295,0.75,3490,0,0.040000,35420,29,0,19.913559,166,70,...,112,7,0,0,0.000026,0.028777,8,86,39,False
59296,0.75,8799,0,0.040000,19249,29,0,19.913559,160,34,...,47,35,0,0,0.000026,0.028777,8,86,125,False
59297,1.00,12795,0,0.040000,40327,10,0,19.913559,166,377,...,79,35,0,0,0.000026,0.028777,8,86,134,False


In [571]:
df_train.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,customer_position,response_corporate,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,9070,0,0.066667,32160,10,0,19.913559,420,62,...,47,33,1,0,0.003079,0.026846,0,28,0,True
1,1.0,8406,0,0.066667,23122,10,0,12.000000,303,96,...,21,33,1,0,0.003079,0.026846,0,0,1,True
2,1.0,6535,0,0.088889,1755,10,0,144.000000,160,56,...,80,21,1,0,0.003079,0.026846,0,17,2,True
3,1.0,3388,0,0.088889,4919,10,0,19.913559,166,44,...,21,21,1,0,0.003079,0.026846,0,44,3,True
4,1.0,5799,0,0.088889,17126,29,0,19.913559,84,97,...,80,21,0,0,0.003079,0.026846,0,86,4,True


In [572]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

## 3. 모델 학습

### 모델 정의 

In [573]:
model = DecisionTreeClassifier()
# model = RandomForestClassifier()

In [574]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))


In [575]:
model.fit(x_train.fillna(0), y_train)

DecisionTreeClassifier()

In [576]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  766   238]
 [  233 10622]]

정확도: 0.9603
정밀도: 0.7668
재현율: 0.7629
F1: 0.7649


### GridSearchCV를 이용한 하이퍼파라미터 튜닝

In [577]:
# 탐색할 하이퍼파라미터 설정
param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
grid_search.fit(x_train.fillna(0), y_train)

# 최적의 하이퍼파라미터 출력
print('최고 평균 정확도 수치 : {0:.4f}'.format(grid_search.best_score_))
print("Best Hyperparameters:", grid_search.best_params_)

최고 평균 정확도 수치 : 0.7580
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 2, 'min_samples_split': 2}


In [578]:
best_tree = grid_search.best_estimator_

### 모델 학습

In [579]:
# model.fit(x_train.fillna(0), y_train)
#
# rf_clf1 = RandomForestClassifier(n_estimators = 100,
#                                  max_depth = 12,
#                                  random_state = 0,)
# rf_clf1.fit(x_train.fillna(0), y_train)
# pred = rf_clf1.predict(x_val.fillna(0))
# print(classification_report(y_val, pred))

### 모델 성능 보기

In [580]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [581]:
# pred = model.predict(x_val.fillna(0))
# pred = rf_clf1.predict(x_val.fillna(0))
pred = best_tree.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  721   283]
 [  141 10714]]

정확도: 0.9642
정밀도: 0.8364
재현율: 0.7181
F1: 0.7728


In [582]:
# model.fit(x_train.fillna(0),y_train)

## 4. 제출하기

In [583]:
# # 최적의 모델로 예측
# best_dt_model = grid_search.best_estimator_
# y_pred = best_dt_model.predict(x_val.fillna(0))
#
# # 성능 평가
# print("\nClassification Report:")
# print(classification_report(y_val, y_pred))

### 테스트 데이터 예측

In [584]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [585]:
# test_pred = model.predict(x_test.fillna(0))
# test_pred = rf_clf1.predict(x_test.fillna(0))
test_pred = best_tree.predict(x_test.fillna(0))

sum(test_pred) # True로 예측된 개수

608

### 제출 파일 작성

In [586]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**